# Load database

This notebook will prepare a combined database using different sources of data. At the end, a .csv file will be created containg two columns: tweet- containing the text, ironic- containing the sentiment.

In [9]:
# General import and load data
import numpy as np
import nltk
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize, word_tokenize
import re

# Needed for running
nltk.download('punkt')
nltk.download('stopwords')

# Import database (mejicano and TASS)
df_noironicos=pd.read_csv('DATA/noironicos_bodies.csv')
df_train = pd.read_csv('DATA/train_data.csv')
df_development = pd.read_csv('DATA/development.csv')


# Encode categorical variable (ironic)
#df_noironicos.loc[df_noironicos['ironic']==True, "ironic"] = 1
#df_noironicos.loc[df_noironicos['ironic']==False, "ironic"] = 0
df_train.loc[df_train['sentiment/polarity/value']== 'NEU', 'sentiment/polarity/value'] = 0
df_train.loc[df_train['sentiment/polarity/value']== 'NONE', 'sentiment/polarity/value'] = 0
df_train.loc[df_train['sentiment/polarity/value']== 'N', 'sentiment/polarity/value'] = 1
df_train.loc[df_train['sentiment/polarity/value']== 'P', 'sentiment/polarity/value'] = 1
df_development.loc[df_development['sentiment'] == 'NONE', 'sentiment'] = 0
df_development.loc[df_development['sentiment'] == 'NEU', 'sentiment'] = 0
df_development.loc[df_development['sentiment'] == 'N', 'sentiment'] = 1
df_development.loc[df_development['sentiment'] == 'P', 'sentiment'] = 1



# Drop non-used columns
df_noironicos.drop(['id_tweet', 'depends_image', 'depends_link', 'depends_retweet'], axis=1, inplace=True)
df_train.drop(['tweetid', 'user', 'date', 'lang'], axis=1, inplace=True)
df_development.drop(['tweetid', 'user', 'date', 'lang'], axis=1, inplace=True)

#
# Drop nan rows
df_clean=df_noironicos.dropna(subset=['text'])
df_noironicos=df_clean

# Final dataset
df_noironicos

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


ironic                                               text
0      True  Algunas personas sufren en las discos mientras...
3      True  @jacevedoaraya es para sostener el marcador......
4      True  Alguna de estas imágenes te sacara una sonrisa...
5      True  @_Eurovision2014 en 2013 falta esdm jajajajaja...
6      True            Hooo que buen padre...#Sarcasmo #GH2015
7      True  @JhoynerV  ja ja ja ja ja así o más claro, cas...
8     False  @patronbermudez con todo respeto lo principios...
11     True  Gran rapidez todo en la UPO y no iban a ser me...
12     True  ¿Que humilde es Simeone no? #ironía #llorón #f...
13     True  ¿Alguien se viene a la playa conmigo? #resfria...
14    False  Tantos enamorados que no son novios😵💑💔💓\ny tan...
15     True  Despues de 4 años de desastres políticos, el ú...
16     True  @CHENCHA_MUU Por eso lo dije. Pq no hay agua. ...
17     True  @RossyFloresR @caacosta1962 antes solo pelucon...
18     True  Igual, igual que @Pablo_Iglesias_  \n#Ironia  ...
19    False  Dia de mi cumpleaños todos me hablan, dia sigu...
20     True  Le comentaré receptor ingreso q ya nos avisare...
22    False  Escuchar cantar a esos que tienen audífonos y ...
23     True  “@cspatria: Termina el partido en el Samborond...
25     True  "Que yo estoy enamorao' prusss yo estoy es tra...
26     True       Ironías de la vida!! https://t.co/QPfPUJqoFE
27     True  @victor_gonzalo_ @joanconti i ara Unió si treu...
28     True  Un sábado de pactos y celebracion #politica po...
29     True  Seré la única que piensa en que con todos esos...
30     True  Le hicieron creer a todos que era la vesícula....
31     True  @notihoyweb creo que les falta un poquito de i...
32     True  Yo sé, pero te dije que pelearíamos 😁 https://...
33     True  #SoyFelizCuando se a donde voy. La #ironia es ...
34     True  Es de idiotas que te enteres que esa persona y...
37     True  7 mil millones de personas y tu insistes en al...
...     ...                                                ...
6299   True  "Qué bien que juega #BRA . No lo deja jugar a ...
6300   True                       La montaste cabron #SARCASMO
6301   True  @Casal @AnaPerezMartin ¡Anda! ¡Y yo que creía ...
6302   True  @danielordas_es ahí esta! Q se acabe el mundo....
6304   True  Que poco twittea la gente a estas horas de la ...
6305   True  @minhothefrog El Chiqui×2 mola mogollón lo bai...
6306   True  Asquerosas barras de Toloun, a la cárcel 100 a...
6307   True  #Sarcasmo\nNueva nada más, mira nada más...\n#...
6309   True  Toma ya, que forma mas discreta de copiar un t...
6311  False  "@sabiasundato_o: Investigación sugiere que la...
6312   True  Ah! Qué no se me había perdido el móvil, desde...
6314   True  @LeMrRobespierre Y guillotine usted a gusto, h...
6315   True  Eso no fue foul, no porque no fue foul😌 #sarca...
6316   True  @JuanMalaga2011 esto además de ilegal debería ...
6318   True  No manchen! A poco Televisa y TVAzteca les pas...
6319   True  @SarponRMCF @rauL_VCF1919 @partidodelas12 Entr...
6320   True  Argumentos férreos, sólidos, dignos de un cate...
6321   True  @tapito72 jajajajaja me faltó el #sarcasmo. Sa...
6322   True  Si alguien tiene semillas de acetaminofén por ...
6323   True  Para decir q votaron 10 millones?o aun envían ...
6325   True  Diseño de sonrisas. #ventanas #slp #andarespot...
6328   True  @Sandris_28 como no en bikini👙 y grande Lola #...
6331   True  La #naturaleza no nos invade nuestro espacio, ...
6334   True  Ultima hora! En Asamblea de la LNFG se decide ...
6335   True  @JuanSoler_ igualmente Majo! #sarcasmo\n\n#A5L...
6337   True  #Sábado "Cría cuervos y te sacaran los ojos" #...
6339   True  Con este pinche calor y uds quieren hacer veda...
6340   True  No te simpatizo y aqui estas stalkeandome.\n#I...
6341   True  MIRÁ QUE BIEN LA #OPOSICION ,YA ESTÁN CONVOCAN...
6344   True  @vicfanfatal @gaspar_hispano ¡Claro que se pue...

[4864 rows x 2 columns]

In [10]:
# import more data from pickle. Additional datasets for balancing.
import pickle


# 1st dataset
train17 = open("DATA/CLASSIFIERS/train17.pickle", "rb")
train17 = pickle.load(train17)
for tweet in train17:
    del(tweet[0])
    del(tweet[0])

# 2nd dataset    
dev17 = open("DATA/CLASSIFIERS/dev17.pickle", "rb")
dev17 = pickle.load(dev17)
for tweet in dev17:
    del(tweet[0])
    del(tweet[0])

# 3rd dataset    
train15 = open("DATA/CLASSIFIERS/train15.pickle", "rb")
train15 = pickle.load(train15)
for tweet in train15:
    del(tweet[0])
    del(tweet[0])




In [11]:
# 4th dataset    
train15_2 = open("DATA/CLASSIFIERS/train15_2.pickle", "rb")
train15_2 = pickle.load(train15_2)
for tweet in train15_2:
    del(tweet[0])
    del(tweet[0])


In [12]:
len(train15_2)+ len(train15)+len(dev17)+len(train17)

13453

In [13]:
tot_lists = train17+dev17+train15+train15_2
len(tot_lists)

13453

In [14]:
# Merge all tweets and create a common dataframe
# df_train and df_development are the TASS datasets. df_noironicos is the mexican dataset containing ironic tweets

sentiment = []
field = []
i=0
for tweet in tot_lists:
    sentiment.append(tweet[1])
    field.append(tweet[0])
    i = i+1
    
# Create a list for every column in dataset
field_1 = df_noironicos['text'].tolist()
sentiment_1 = df_noironicos['ironic'].tolist()
sentiment_2 = df_development['sentiment'].tolist()
field_2 = df_development['content'].tolist()
field_3 = df_train['content'].tolist()
sentiment_3 = df_train['sentiment/polarity/value'].tolist()

# Merge lists and define dataset
field_complete = field_1+field_2+field_3+field
sentiment_complete = sentiment_1+sentiment_2+sentiment_3+sentiment
df = pd.DataFrame(
    {'tweet': field_complete,
     'ironic': sentiment_complete
    })

In [27]:
# COdify all non-numeric variables

df.loc[df['ironic'] == 'NONE', 'ironic'] = 0
df.loc[df['ironic'] == 'NEU', 'ironic'] = 0
df.loc[df['ironic'] == 'N', 'ironic'] = 2
df.loc[df['ironic'] == 'P', 'ironic'] = 2
df.loc[df['ironic'] == True, 'ironic'] = 1
df.loc[df['ironic'] == False, 'ironic'] = 0
df = df[df.ironic != 2]
df = df[df.ironic != 'N+']
df = df[df.ironic != 'P+']


In [28]:
# Save dataset

df.to_csv('final_dataset.csv', index = False)